In [7]:
!pip install numpy music21 tensorflow

In [8]:
!wget https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0-midi.zip -O /content/maestro-v3.0.0-midi.zip

--2025-08-20 12:16:47--  https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0-midi.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.187.207, 64.233.188.207, 64.233.189.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.187.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58416533 (56M) [application/octet-stream]
Saving to: ‘/content/maestro-v3.0.0-midi.zip’

/content/maestro-v3 100%[===================>]  55.71M  19.0MB/s    in 2.9s    

2025-08-20 12:16:50 (19.0 MB/s) - ‘/content/maestro-v3.0.0-midi.zip’ saved [58416533/58416533]



In [9]:
!unzip /content/maestro-v3.0.0-midi.zip -d /content/maestro


Archive:  /content/maestro-v3.0.0-midi.zip
  inflating: /content/maestro/maestro-v3.0.0/2004/MIDI-Unprocessed_XP_08_R1_2004_01-02_ORIG_MID--AUDIO_08_R1_2004_01_Track01_wav.midi  
  inflating: /content/maestro/maestro-v3.0.0/2004/MIDI-Unprocessed_XP_09_R1_2004_05_ORIG_MID--AUDIO_09_R1_2004_06_Track06_wav.midi  
  inflating: /content/maestro/maestro-v3.0.0/2004/MIDI-Unprocessed_XP_14_R1_2004_01-03_ORIG_MID--AUDIO_14_R1_2004_01_Track01_wav.midi  
  inflating: /content/maestro/maestro-v3.0.0/2004/MIDI-Unprocessed_XP_01_R1_2004_01-02_ORIG_MID--AUDIO_01_R1_2004_03_Track03_wav.midi  
  inflating: /content/maestro/maestro-v3.0.0/2004/MIDI-Unprocessed_SMF_13_01_2004_01-05_ORIG_MID--AUDIO_13_R1_2004_09_Track09_wav.midi  
  inflating: /content/maestro/maestro-v3.0.0/2004/MIDI-Unprocessed_XP_18_R1_2004_01-02_ORIG_MID--AUDIO_18_R1_2004_03_Track03_wav.midi  
  inflating: /content/maestro/maestro-v3.0.0/2004/MIDI-Unprocessed_XP_19_R1_2004_01-02_ORIG_MID--AUDIO_19_R1_2004_01_Track01_wav.midi  
  infla

In [11]:
import numpy as np
from music21 import converter, instrument, note, chord, stream
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
import os
import glob
import pickle

def get_notes(midi_files):
    notes = []
    total_files = len(midi_files)
    for i, file in enumerate(midi_files):
        print(f"Parsing file {i+1}/{total_files}: {os.path.basename(file)}")
        try:
            midi = converter.parse(file)
            parts = instrument.partitionByInstrument(midi)
            notes_to_parse = None
            if parts:
                notes_to_parse = parts.parts[0].recurse()
            else:
                notes_to_parse = midi.flat.notes

            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))
        except Exception as e:
            print(f"Error parsing {file}: {e}")

    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

def prepare_sequences(notes, sequence_length=100):
    pitch_names = sorted(set(notes))
    n_vocab = len(pitch_names)

    with open('data/pitch_names', 'wb') as filepath:
        pickle.dump(pitch_names, filepath)

    note_to_int = {note: number for number, note in enumerate(pitch_names)}

    network_input = []
    network_output = []

    for i in range(0, len(notes) - sequence_length):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    network_input = network_input / float(n_vocab)
    network_output = to_categorical(network_output, num_classes=n_vocab)

    return network_input, network_output

def create_model(network_input, n_vocab):
    model = Sequential()
    model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dropout(0.3))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(n_vocab, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

def train_model(network_input, network_output, n_vocab):
    model = create_model(network_input, n_vocab)

    # NEW LINE
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}.keras"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=50, batch_size=128, callbacks=callbacks_list, verbose=1)
    return model

def main():
    DATASET_PATH = "/content/maestro/maestro-v3.0.0/**/*.midi"
    MAX_FILES_TO_PROCESS = 10

    if not os.path.exists('data'):
        os.makedirs('data')

    if os.path.exists('data/notes'):
        print("Loading notes from cache...")
        with open('data/notes', 'rb') as filepath:
            notes = pickle.load(filepath)
    else:
        print("Parsing MIDI files...")
        midi_files = glob.glob(DATASET_PATH, recursive=True)
        if not midi_files:
            print(f"No MIDI files found at '{DATASET_PATH}'. Please check the path.")
            return

        midi_files = midi_files[:MAX_FILES_TO_PROCESS]

        notes = get_notes(midi_files)

    if not notes:
        print("No notes extracted. Aborting.")
        return

    n_vocab = len(set(notes))
    print(f"Total notes parsed: {len(notes)}")
    print(f"Vocabulary size (unique notes/chords): {n_vocab}")

    sequence_length = 10
    network_input, network_output = prepare_sequences(notes, sequence_length)

    print("Building and training model...")
    train_model(network_input, network_output, n_vocab)

    print("\nTraining complete. To generate music, run a separate generation script using the saved weights.")

if __name__ == "__main__":
    main()

Loading notes from cache...
Total notes parsed: 22582
Vocabulary size (unique notes/chords): 818
Building and training model...
Epoch 1/50


KeyboardInterrupt: 

In [12]:
import numpy as np
from music21 import converter, instrument, note, chord, stream
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import pickle
import glob
import os

def create_model(network_input, n_vocab):
    """ Recreate the exact same model architecture as in training """
    model = Sequential([
        LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True),
        Dropout(0.3),
        LSTM(512),
        Dropout(0.3),
        Dense(256, activation='relu'),
        Dense(n_vocab, activation='softmax')
    ])
    # No need to compile for prediction, but it's good practice
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

def generate_notes(model, network_input, pitch_names, n_vocab, num_notes=200):
    """ Generate notes from the neural network based on a random seed """
    start = np.random.randint(0, len(network_input)-1)
    int_to_note = {number: note for number, note in enumerate(pitch_names)}

    # Use a slice of the original input data as the starting pattern
    pattern_indices = network_input[start]
    prediction_output = []

    # --- THIS IS THE FIX ---
    # Convert the list to a numpy array before performing division
    pattern = np.array(pattern_indices) / float(n_vocab)

    for note_index in range(num_notes):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        # Update the pattern: remove the first note and add the new prediction
        new_index_normalized = index / float(n_vocab)
        pattern = np.append(pattern[1:], new_index_normalized)

    return prediction_output

def create_midi(prediction_output, output_file='ai_generated_music.mid'):
    """ Convert the output from the prediction to notes and create a midi file """
    offset = 0
    output_notes = []
    for pattern in prediction_output:
        # It's a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            chord_notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                chord_notes.append(new_note)
            new_chord = chord.Chord(chord_notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # It's a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        # Increment offset for the next note/chord
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp=output_file)
    print(f"Successfully saved generated music to {output_file}")

def main():
    print("Loading data for music generation...")
    with open('data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)
    with open('data/pitch_names', 'rb') as filepath:
        pitch_names = pickle.load(filepath)

    n_vocab = len(set(notes))
    sequence_length = 100 # Must be the same as in training

    # Prepare sequences for seeding the model (we only need the input part)
    note_to_int = {note: number for number, note in enumerate(pitch_names)}
    network_input = []
    for i in range(len(notes) - sequence_length):
        sequence_in = notes[i:i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])

    # Find the best weights file
    weight_files = glob.glob("weights-improvement-*.keras")
    if not weight_files:
        print("Error: No weights files found. Please run train.py first.")
        return

    # Sort files by epoch number to get the latest one
    latest_weights_file = max(weight_files, key=os.path.getctime)
    print(f"Loading weights from: {latest_weights_file}")

    # Recreate the model and load the weights
    model_input_shape = (len(network_input[0]), 1) # (sequence_length, 1)
    model = create_model(np.zeros((1, *model_input_shape)), n_vocab)
    model.load_weights(latest_weights_file)

    print("Generating new music...")
    prediction_output = generate_notes(model, network_input, pitch_names, n_vocab)

    print("Converting generated notes to MIDI file...")
    create_midi(prediction_output)

if __name__ == "__main__":
    main()

Loading data for music generation...
Loading weights from: weights-improvement-01-5.5627.keras


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Generating new music...
Converting generated notes to MIDI file...
Successfully saved generated music to ai_generated_music.mid


In [13]:
# Install the synthesizer program (example for Linux/Colab)
!sudo apt-get install fluidsynth
!pip install midi2audio
# Install the Python library
!pip install pyfluidsynth

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fluidsynth is already the newest version (2.2.5-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [14]:
# Add this import at the top
from midi2audio import FluidSynth

# After calling create_midi(), add this:
print("Converting MIDI to audio...")
fs = FluidSynth('path/to/your/GeneralUser_GS_v1.471.sf2') # Use the path to your .sf2 file
fs.midi_to_audio('ai_generated_music.mid', 'ai_generated_music.wav')
print("Audio file ai_generated_music.wav saved successfully!")

Converting MIDI to audio...
Audio file ai_generated_music.wav saved successfully!
